In [3]:
import pandas as pd
import numpy as np
import lightgbm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import math
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
#открываем train, заполняем пустоты нулями (это оказалось лучшей стратегией)
train = pd.read_csv('train2.csv', encoding='latin-1', parse_dates=['Date', 'OrderDate'])
train = train.fillna(0.0)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
#открываем test полный, также заполняем пустоты нулями
test_all = pd.read_csv('test2_old.csv', encoding='latin-1', parse_dates=['Date', 'OrderDate'])
test_all = test_all.fillna(0.0)

In [7]:
#для последующих итерационных отладок
train_save = train
test_all_save = test_all

In [359]:
#для последующих итерационных отладок
train = train_save
test = test_all_save

In [360]:
#train_dop1 = train.groupby(by=['OrderID', 'GroupID'])['OrderCnt'].agg(["sum"]).unstack(level=-1).reset_index().fillna(0)
#train_dop2 = train.groupby(by=['OrderID', 'GroupID'])['ConfCnt'].agg(["sum"]).unstack(level=-1).reset_index().fillna(0)
train_dop3 = train.groupby(by=['OrderID'])['OrderCnt'].agg(["sum"]).reset_index()
train_dop4 = train.groupby(by=['OrderID'])['ConfCnt'].agg(["sum"]).reset_index()
train_dop5 = train.groupby(by=['OrderID'])['OrderID'].agg(["count"]).reset_index()

#test_dop1 = test.groupby(by=['OrderID', 'GroupID'])['OrderCnt'].agg(["sum"]).unstack(level=-1).reset_index().fillna(0)
#test_dop2 = test.groupby(by=['OrderID', 'GroupID'])['ConfCnt'].agg(["sum"]).unstack(level=-1).reset_index().fillna(0)
test_dop3 = test.groupby(by=['OrderID'])['OrderCnt'].agg(["sum"]).reset_index()
test_dop4 = test.groupby(by=['OrderID'])['ConfCnt'].agg(["sum"]).reset_index()
test_dop5 = test.groupby(by=['OrderID'])['OrderID'].agg(["count"]).reset_index()

In [361]:
train = train.groupby(by=['OrderID', 'Interval', 'Date', 'OrderDate', 'ClientID', 'ChannelID', 'Cluster', 'DeliveryType', 'prepay', 'count_edit'])['Delta'].agg(["mean"]).reset_index()

In [362]:
test = test.groupby(by=['OrderID', 'Interval', 'Date', 'OrderDate', 'ClientID', 'ChannelID', 'Cluster', 'DeliveryType', 'prepay', 'count_edit'])['prepay'].agg(["count"]).reset_index()

In [363]:
train['Delta'] = train['mean']
train = train.drop(['mean'], axis=1)
test = test.drop(['count'], axis=1)

In [364]:
#train = train.merge(train_dop1, on=['OrderID'], how='left')
#train = train.merge(train_dop2, on=['OrderID'], how='left')
train = train.merge(train_dop3, on=['OrderID'], how='left')
train = train.merge(train_dop4, on=['OrderID'], how='left')
train = train.merge(train_dop5, on=['OrderID'], how='left')

#test = test.merge(test_dop1, on=['OrderID'], how='left')
#test = test.merge(test_dop2, on=['OrderID'], how='left')
test = test.merge(test_dop3, on=['OrderID'], how='left')
test = test.merge(test_dop4, on=['OrderID'], how='left')
test = test.merge(test_dop5, on=['OrderID'], how='left')

In [365]:
train['cnt_delta'] = train.sum_y/train.sum_x
test['cnt_delta'] = test.sum_y/test.sum_x

In [366]:
train = train.sort_values(by=['ClientID', 'OrderDate', 'OrderID'])
train['ttt'] = train[['ClientID', 'OrderDate']].duplicated(keep='last')
train['ttt'] = train.ttt.apply(lambda x: 1 if (x == True) else 0)
train = train.sort_values(by=['OrderID'])

In [367]:
test = test.sort_values(by=['ClientID', 'OrderDate', 'OrderID'])
test['ttt'] = test[['ClientID', 'OrderDate']].duplicated(keep='last')
test['ttt'] = test.ttt.apply(lambda x: 1 if (x == True) else 0)
test = test.sort_values(by=['OrderID'])

In [368]:
#для обучения нам не нужны безотказные заказы
train = train[train.Delta != 0]

In [369]:
#доп.признаки (не все они понадобятся в итоге)
train['DeliveryType_int'] = train.DeliveryType.apply(lambda x: 0 if (x == 'Îáû÷íàÿ äîñòàâêà') else 1)
train['Date_dd'] = train.Date.dt.day
train['Date_mm'] = train.Date.dt.month
train['Date_y'] = train.Date.dt.year
train['Date_weekday'] = train.Date.dt.weekday
train['Date_dy'] = train.Date.dt.dayofyear
train['OrderDate_dd'] = train.OrderDate.dt.day
train['OrderDate_mm'] = train.OrderDate.dt.month
train['OrderDate_y'] = train.OrderDate.dt.year
train['OrderDate_weekdate'] = train.OrderDate.dt.weekday
train['OrderDate_dy'] = train.OrderDate.dt.dayofyear
train['delta_days'] = train.Date - train.OrderDate
train['delta_days'] = (24.0*train['delta_days'].astype(np.int64)/86400000000000).astype(np.int64)

In [370]:
#аналогичные доп.признаки для test
test['DeliveryType_int'] = test.DeliveryType.apply(lambda x: 0 if (x == 'Îáû÷íàÿ äîñòàâêà') else 1)
test['Date_dd'] = test.Date.dt.day
test['Date_mm'] = test.Date.dt.month
test['Date_y'] = test.Date.dt.year
test['Date_weekday'] = test.Date.dt.weekday
test['Date_dy'] = test.Date.dt.dayofyear
test['OrderDate_dd'] = test.OrderDate.dt.day
test['OrderDate_mm'] = test.OrderDate.dt.month
test['OrderDate_y'] = test.OrderDate.dt.year
test['OrderDate_weekdate'] = test.OrderDate.dt.weekday
test['OrderDate_dy'] = test.OrderDate.dt.dayofyear
test['delta_days'] = test.Date - test.OrderDate
test['delta_days'] = (24.0*test['delta_days'].astype(np.int64)/86400000000000).astype(np.int64)

In [371]:
#заэнкодим временные интервалы доставки (много почему-то не дают, но все же)
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(list(set(pd.unique(test.Interval)) | set(pd.unique(train.Interval))))
train['Interval_int'] = le.transform(train['Interval'].values) 
test['Interval_int'] = le.transform(test['Interval'].values) 

In [372]:
#уберем предновогодние выбросы <180 (в тесте нет предновогоднего периода)
train = train[train.Delta < 180]

In [373]:
#Создаем массив признаков, которые не пойдут в модель
KILL_COLS = [
    'Interval', 'sum_x', 
    'Date', 
    'OrderDate', 
    'DeliveryType', 
    'Cluster', 
    'Delta', 
    'Date_y', 
    'OrderDate_y',
    'Date_dy',
    'OrderDate_dy'
]

In [374]:
#такой же массив для тестового набора
KILL_COLS_TEST = [
    'Interval', 'sum_x',
    'Date', 
    'OrderDate', 
    'DeliveryType', 
    'Cluster', 
    'Date_y', 
    'OrderDate_y',
    'Date_dy',
    'OrderDate_dy'
]

# Train_test_split (отладочный блок)

In [348]:
#сделаем примитивную разбивку на train/test без шаффла
train_part, validation = train_test_split(train, test_size=0.33, shuffle=False)

In [350]:
#13.728298224967453
lgb = lightgbm.LGBMRegressor(random_state=0)
lgb.fit(train_part.drop(KILL_COLS, axis = 1).values, train_part['Delta'].values)

pred_lgb = lgb.predict(validation.drop(KILL_COLS, axis = 1).values)
print(math.sqrt(mean_squared_error(validation['Delta'].values, pred_lgb)))

26.2650601679974


In [331]:
#13.767881025653487
GBoost = GradientBoostingRegressor(n_estimators=320, learning_rate=0.04,
                                   max_depth=5, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, loss='huber', random_state=1)
GBoost.fit(train_part.drop(KILL_COLS, axis = 1).values, train_part['Delta'].values)
pred_GBoost = GBoost.predict(validation.drop(KILL_COLS, axis = 1).values)
print(math.sqrt(mean_squared_error(validation['Delta'].values, pred_GBoost)))

13.746025704242703


In [81]:
lgb.feature_importances_

array([247, 462,  22,  12,  98, 239,  34, 226,  27, 171,  17, 124,  82,
        75, 108,  79,  87, 510, 380], dtype=int32)

# Final model (LGBM + GradientBoostingRegressor)

In [375]:
#устраним последствия деления на ноль (деление количества подтвержденных позиций на колчиество заказанных - см.выше в блоке подготовки доп.признаков)
test=test.fillna(0.0)

In [376]:
lgb = lightgbm.LGBMRegressor(random_state=0, n_estimators=110)
lgb.fit(train.drop(KILL_COLS, axis=1).values, train['Delta'].values)
#lgb.fit(train.drop(KILL_COLS, axis=1).values, (train[["Delta"]].values - train["Delta"].mean())/(train["Delta"].std()))
pred_lgb = lgb.predict(test.drop(KILL_COLS_TEST, axis=1).values)
#pred_lgb = lgb.predict(test.drop(KILL_COLS_TEST, axis=1).values)*train["Delta"].std() + train["Delta"].mean()

In [914]:
temp = pd.concat([test.OrderID, pd.DataFrame(pred_lgb, columns=['Score'])], axis = 1)

In [383]:
GBoost = GradientBoostingRegressor(n_estimators=330, learning_rate=0.04,
                                   max_depth=5, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, loss='huber', random_state=1)
GBoost.fit(train.drop(KILL_COLS, axis=1).values, train['Delta'].values)
pred_GBoost = GBoost.predict(test.drop(KILL_COLS_TEST, axis=1).values)

In [422]:
temp = pd.concat([test.OrderID, pd.DataFrame(pred_GBoost, columns=['Score'])], axis = 1)

In [384]:
#для смешивания результатов 
temp = pd.concat([test.OrderID, pd.DataFrame((pred_GBoost+pred_lgb)/2.0, columns=['Score'])], axis = 1)

## Генерируем файл ответа

In [385]:
otvet = pd.read_csv('empty2.csv')
otvet = otvet.drop([' Score'], axis=1)

In [386]:
temp['ID'] = temp.OrderID
temp[' Score'] = temp.Score

In [387]:
otvet=otvet.merge(temp, on=['ID'], how='left').drop(['Score', 'OrderID'], axis=1)

In [388]:
otvet.to_csv('Submit.csv', index=False)